In [365]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from ml4comm.qam_awgn import generate_symbols
from ml4comm.qam_analyzer import plot_decision_boundary, ser, plot_confusion_matrix
from ml4comm.plot import plot_symbols
from ml4comm.symbols import setup_channel, generate_dataset

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [366]:
channel_type = 'crazy' # 'awgn' or 'crazy'
M            = 16      # QAM modulation
crazy_train_size = 600
awgn_train_size = 450
num_symbols  = 3000 + (awgn_train_size if channel_type == 'awgn' else crazy_train_size)    # Number of transmitted symbols
SNR_dB       = 15      # Signal to noise ratio in dB     
code_rate    = 1       # Rate of the used code
Es           = 1       # Average symbol energy

NUM_THREADS=-1 # use all available threads
MAX_ERROR_RATE = 0.03 if channel_type == 'awgn' else 0.02 # Max error rate: 3%

# Model Parameters
RF_MAX_DEPTH      = 50
RF_NUM_ESTIMATORS = 50
KNN_NUM_NEIGHBORS = 11

In [367]:
def classification_workflow(model):
  symbs, indexes = generate_symbols(num_symbols, M) # Generate the QAM symbols
  channel_output = setup_channel(channel_type,symbs,code_rate,Es,SNR_dB)
  [X_train, X_test, y_train, y_test] = generate_dataset(channel_output,indexes)
  clf = model.fit(X_train, y_train) 
  return ser(clf, X_test, y_test)

def print_accuracy(ser_array: np.ndarray, threshold: float):
  arr = np.where(ser_array <= threshold, 1, 0)
  output = np.sum(arr)/ser_array.size

  print('-----Resumo-----')
  for index in range(0, ser_array.size):
    print(f'Rodada {index} | {ser_array[index]:.2%}')

  print(f'\nResultados dentro do limite de {MAX_ERROR_RATE:.2%}: {output:.2%}')
  

In [368]:
symbs, indexes = generate_symbols(num_symbols, M) # Generate the QAM symbols
channel_output = setup_channel(channel_type,symbs,code_rate,Es,SNR_dB)
[X_train, X_test, y_train, y_test] = generate_dataset(channel_output,indexes)

In [369]:
# plot_symbols(X_train, y_train, symbs, QAM_modulation=M)

## Classifier

### K-nearest neighbors

In [370]:
clf_KNN = KNeighborsClassifier(KNN_NUM_NEIGHBORS, n_jobs=NUM_THREADS)
sers = [classification_workflow(clf_KNN) for x in range(33)]
sers = np.array(sers)
print_accuracy(sers, MAX_ERROR_RATE)
# plot_decision_boundary(clf_KNN, X_train, y_train, legend=True)
# plot_confusion_matrix(clf_KNN, X_test, y_test, M)

-----Resumo-----
Rodada 0 | 1.53%
Rodada 1 | 1.20%
Rodada 2 | 1.20%
Rodada 3 | 1.43%
Rodada 4 | 1.67%
Rodada 5 | 1.53%
Rodada 6 | 1.53%
Rodada 7 | 1.50%
Rodada 8 | 1.60%
Rodada 9 | 1.30%
Rodada 10 | 1.60%
Rodada 11 | 1.27%
Rodada 12 | 1.50%
Rodada 13 | 1.27%
Rodada 14 | 1.57%
Rodada 15 | 1.70%
Rodada 16 | 2.00%
Rodada 17 | 1.17%
Rodada 18 | 1.07%
Rodada 19 | 1.57%
Rodada 20 | 1.50%
Rodada 21 | 1.57%
Rodada 22 | 1.37%
Rodada 23 | 1.17%
Rodada 24 | 1.57%
Rodada 25 | 1.57%
Rodada 26 | 1.40%
Rodada 27 | 1.30%
Rodada 28 | 1.60%
Rodada 29 | 1.37%
Rodada 30 | 1.53%
Rodada 31 | 1.50%
Rodada 32 | 2.03%

Resultados dentro do limite de 2.00%: 96.97%
